# <center> 流畅的Python </center>
## 第十三章：正确重载运算符
**示例13-1 把一元运算符添加到示例10-16中**

In [1]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if(isinstance(index, slice)):
            return cls(self._components[index])
        elif(isinstance(index, numbers.Integral)):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if(len(name) == 1):
            pos = cls.shortcut_names.find(name)
            if(0 <= pos < len(self._components)):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if(len(name) == 1):
            if(name in cls.shortcut_names):
                error = 'readonly attribute {attr_name!r}'
            elif(name.islower()):
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if(error):
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if((n == len(self) - 1) and (self[-1] < 0)):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if(fmt_spec.endswith('h')):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            out_fmt = '<{}>'
        else:
            coords = self
            out_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return out_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

**示例13-2 算术运算上下文的精度变化可能导致x不等于+x**

In [2]:
import decimal
ctx = decimal.getcontext()
ctx.prec = 10
one_third = decimal.Decimal('1') / decimal.Decimal('3')
print(one_third)
print(one_third == + one_third)
ctx.prec = 5
print(one_third == + one_third)
print(+ one_third)

0.3333333333
True
False
0.33333


**示例13-3 一元运算符+得到一个新Counter实例，但是没有零值和负值计算器**

In [3]:
from collections import Counter

ct = Counter('abracadabra')
print(ct)
ct['r'] = -3
ct['d'] = 0
print(ct)
print(+ ct)


Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})
Counter({'a': 5, 'b': 2, 'c': 1})


**示例13-4 Vector.\__add\__方法，第一版**

In [4]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a, b in pairs)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if(isinstance(index, slice)):
            return cls(self._components[index])
        elif(isinstance(index, numbers.Integral)):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if(len(name) == 1):
            pos = cls.shortcut_names.find(name)
            if(0 <= pos < len(self._components)):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if(len(name) == 1):
            if(name in cls.shortcut_names):
                error = 'readonly attribute {attr_name!r}'
            elif(name.islower()):
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if(error):
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if((n == len(self) - 1) and (self[-1] < 0)):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if(fmt_spec.endswith('h')):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            out_fmt = '<{}>'
        else:
            coords = self
            out_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return out_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

**测试**

In [7]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
print(v1 + v2)
print(v1 + v2 == Vector([9, 11, 13]))

v1 = Vector([3, 4, 5, 6])
v2 = Vector([1, 2])
print(v1 + v2)

(9.0, 11.0, 13.0)
True
(4.0, 6.0, 5.0, 6.0)


**示例13-5 第一版方法也支持Vector之外的对象**

In [9]:
v1 = Vector([3, 4, 5])
print(v1 + (10, 20, 30))

(13.0, 24.0, 35.0)


**示例13-6 如果左操作数是Vector之外的对象第一版无法处理**

In [10]:
v1 = Vector([3, 4, 5])
print((10, 20, 30) + v1)

TypeError: can only concatenate tuple (not "Vector") to tuple

**示例13-7 Vector.\__add\__和\__radd\__方法**

In [11]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a, b in pairs)
    
    def __radd__(self, other):
        return self + other
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if(isinstance(index, slice)):
            return cls(self._components[index])
        elif(isinstance(index, numbers.Integral)):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if(len(name) == 1):
            pos = cls.shortcut_names.find(name)
            if(0 <= pos < len(self._components)):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if(len(name) == 1):
            if(name in cls.shortcut_names):
                error = 'readonly attribute {attr_name!r}'
            elif(name.islower()):
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if(error):
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if((n == len(self) - 1) and (self[-1] < 0)):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if(fmt_spec.endswith('h')):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            out_fmt = '<{}>'
        else:
            coords = self
            out_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return out_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

**测试**

In [12]:
v1 = Vector([3, 4, 5])
print((10, 20, 30) + v1)

(13.0, 24.0, 35.0)


**示例13-8 Vector.\__add\__方法的操作数要是可迭代对象**

In [13]:
print(v1 + 1)

TypeError: zip_longest argument #2 must support iteration

**示例13-9 Vector.\__add\__方法的操作数要是可迭代的数值对象**

In [14]:
print(v1 + 'ABC')

TypeError: unsupported operand type(s) for +: 'float' and 'str'

**示例13-10 +运算符方法**

In [15]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if(isinstance(index, slice)):
            return cls(self._components[index])
        elif(isinstance(index, numbers.Integral)):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if(len(name) == 1):
            pos = cls.shortcut_names.find(name)
            if(0 <= pos < len(self._components)):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if(len(name) == 1):
            if(name in cls.shortcut_names):
                error = 'readonly attribute {attr_name!r}'
            elif(name.islower()):
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if(error):
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if((n == len(self) - 1) and (self[-1] < 0)):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if(fmt_spec.endswith('h')):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            out_fmt = '<{}>'
        else:
            coords = self
            out_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return out_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

**示例13-11 增加\*运算符方法**

In [17]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, scalar):
        if(isinstance(scalar, numbers.Real)):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
        
    def __rmul__(self, scalar):
        return self * scalar
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if(isinstance(index, slice)):
            return cls(self._components[index])
        elif(isinstance(index, numbers.Integral)):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if(len(name) == 1):
            pos = cls.shortcut_names.find(name)
            if(0 <= pos < len(self._components)):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if(len(name) == 1):
            if(name in cls.shortcut_names):
                error = 'readonly attribute {attr_name!r}'
            elif(name.islower()):
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if(error):
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if((n == len(self) - 1) and (self[-1] < 0)):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if(fmt_spec.endswith('h')):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            out_fmt = '<{}>'
        else:
            coords = self
            out_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return out_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

**测试**

In [18]:
v1 = Vector([1, 2, 3])
print(14 * v1)
print(v1 * True)

(14.0, 28.0, 42.0)
(1.0, 2.0, 3.0)


**示例13-12 Vector实例与元组比较**

In [19]:
va = Vector([1, 2, 3])
vb = Vector(range(1, 4))
print(va == vb)
print(va == (1, 2, 3))

True
True


**示例13-13 改进\__eq\__方法**

In [26]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        if(isinstance(other, Vector)):
            return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        else:
            return NotImplemented
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, scalar):
        if(isinstance(scalar, numbers.Real)):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
        
    def __rmul__(self, scalar):
        return self * scalar
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if(isinstance(index, slice)):
            return cls(self._components[index])
        elif(isinstance(index, numbers.Integral)):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if(len(name) == 1):
            pos = cls.shortcut_names.find(name)
            if(0 <= pos < len(self._components)):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if(len(name) == 1):
            if(name in cls.shortcut_names):
                error = 'readonly attribute {attr_name!r}'
            elif(name.islower()):
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if(error):
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
        
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if((n == len(self) - 1) and (self[-1] < 0)):
            return math.pi * 2 - a
        else:
            return a
        
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if(fmt_spec.endswith('h')):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            out_fmt = '<{}>'
        else:
            coords = self
            out_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return out_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

**示例13-14 测试**

In [27]:
va = Vector([1, 2, 3])
vb = Vector(range(1, 4))
print(va == vb)
print(va == (1, 2, 3))
print((1, 2, 3) == va)

True
False
False


**示例13-15 增量赋值不会修改不可变目标，而是新建实例，然后重新绑定**

In [28]:
v1 = Vector([1, 2, 3])
v1_alias = v1
print(id(v1))
v1 += Vector([4, 5, 6])
print(v1)
print(id(v1), '\n')

v1 *= 11
print(v1)
print(id(v1))


4552497080
(5.0, 7.0, 9.0)
4552498816 

(55.0, 77.0, 99.0)
4552498144
